In [4]:
import pandas as pd
import numpy as np
import math
import seaborn as sns
import matplotlib.pyplot as plt
import tensorflow as tf
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split
from keras.layers.recurrent import LSTM
from sklearn.metrics import accuracy_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC


In [5]:
#pip install tensorflow --upgrade --user

In [6]:
#pip install tensorflow-hub

In [7]:
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.layers import Dense, Flatten,Conv2D, MaxPooling2D, Dropout, BatchNormalization, TimeDistributed
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam

In [8]:

data1 = pd.read_table('phoneaccel.txt',names=('Subject-id', 'Activity Label', 'Timestamp', 'x', 'y', 'z'),sep=',')
data2 = pd.read_table('phonegyro.txt',names=('Subject-id', 'Activity Label', 'Timestamp', 'x', 'y', 'z'),sep=',')

C:\Users\shiva\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (0) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [9]:
#merging all dataframes
frames = [data1,data2]
data = pd.concat(frames)
data.shape

(6174062, 6)

In [10]:
data.head()

,Subject-id,Activity Label,Timestamp,x,y,z
0,1600,A,2.522077e+14,-0.364761,8.793503,1.0550842;
1,1600,A,2.522077e+14,-0.879730,9.768784,1.0169983;
2,1600,A,2.522078e+14,2.001495,11.109070,2.619156;
3,1600,A,2.522078e+14,0.450623,12.651642,0.18455505;
4,1600,A,2.522079e+14,-2.164352,13.928436,-4.4224854;


In [11]:
#since the datatype of column z is object because of ';', we will now remove the ';' and convert it to float datatype
data['z']= data.z.str.replace(';','').astype('float')
data.dtypes

Subject-id         object
Activity Label     object
Timestamp         float64
x                 float64
y                 float64
z                 float64
dtype: object

In [12]:
#function to return lsit of columns
def colname(df):
    for col in df.columns:
        print(col)

In [13]:
colname(data)

Subject-id
Activity Label
Timestamp
x
y
z


In [14]:
#function to return basic eda
def basic_eda(df):
    print('==========================================ROW===============================================')
    print(df.head())
    print('==========================================ROW===============================================')
    print(df.tail())
    print('==========================================ROW===============================================')
    print(df.info())
    print('==========================================ROW===============================================')
    print(df.dtypes)
    print('==========================================ROW===============================================')
    print(df.describe())
    print('==========================================ROW===============================================')
    print(df.isna().sum())
    print('==========================================ROW===============================================')
    print(df.isnull().sum())
    print('==========================================ROW===============================================')
    print(df.shape)

In [15]:
basic_eda(data)

==========================================ROW===============================================
  Subject-id Activity Label     Timestamp         x          y         z
0       1600              A  2.522077e+14 -0.364761   8.793503  1.055084
1       1600              A  2.522077e+14 -0.879730   9.768784  1.016998
2       1600              A  2.522078e+14  2.001495  11.109070  2.619156
3       1600              A  2.522078e+14  0.450623  12.651642  0.184555
4       1600              A  2.522079e+14 -2.164352  13.928436 -4.422485
==========================================ROW===============================================
        Subject-id Activity Label     Timestamp         x         y         z
1369653       1619              P  1.739443e+14 -0.282295 -0.335558 -0.039415
1369654       1619              P  1.739443e+14 -0.524110 -0.634898 -0.059655
1369655       1619              P  1.739443e+14 -0.532632 -0.763795  0.103331
1369656       1619              P  1.739444e+14 -0.255663 -0.641

In [16]:
#from above eda plot we can see there are 3 null and 3 na values in the respective columns
#now we will remove the na values
data = data.fillna(0,axis=1)
data.isna().sum()

Subject-id        0
Activity Label    0
Timestamp         0
x                 0
y                 0
z                 0
dtype: int64

In [17]:
data['Activity Label'].value_counts()

K    363211
A    359406
M    357054
H    348785
G    347619
E    347610
O    346657
P    346623
L    343800
D    342609
R    341563
I    339372
B    339284
S    338711
Q    333992
C    333655
J    323312
F    320798
0         1
Name: Activity Label, dtype: int64

In [18]:
#dictionary of actual activites
labels = { 'A':'walking' ,'B':'jogging' ,'C':'stairs', 'D':'sitting','E':'standing','F':'typing', 'G':'brushing teeth', 'H':'drinking soup', 'I':'eating chips','J':'eating pasta','K':'drinking', 'L':'eating sandwich', 'M':'kicking','O':'catch','P':'dribbling','Q':'writing','R':'clapping','S':'folding','0':'not recognisable'}

In [19]:
data['Activity Label'].value_counts()

K    363211
A    359406
M    357054
H    348785
G    347619
E    347610
O    346657
P    346623
L    343800
D    342609
R    341563
I    339372
B    339284
S    338711
Q    333992
C    333655
J    323312
F    320798
0         1
Name: Activity Label, dtype: int64

In [20]:
#replacing pseudo activity labels with actual activity labels
data = data.replace({'Activity Label': labels})
data.head()

,Subject-id,Activity Label,Timestamp,x,y,z
0,1600,walking,2.522077e+14,-0.364761,8.793503,1.055084
1,1600,walking,2.522077e+14,-0.879730,9.768784,1.016998
2,1600,walking,2.522078e+14,2.001495,11.109070,2.619156
3,1600,walking,2.522078e+14,0.450623,12.651642,0.184555
4,1600,walking,2.522079e+14,-2.164352,13.928436,-4.422485


In [21]:
activities = data['Activity Label'].value_counts().index
activities

Index([       'drinking',         'walking',         'kicking',
         'drinking soup',  'brushing teeth',        'standing',
                 'catch',       'dribbling', 'eating sandwich',
               'sitting',        'clapping',    'eating chips',
               'jogging',         'folding',         'writing',
                'stairs',    'eating pasta',          'typing',
                       0],
      dtype='object')

In [22]:
#since activity label has just once value as 0 
#to remove it lets find the index number of the specific row
data.loc[data['Activity Label']== 0]

,Subject-id,Activity Label,Timestamp,x,y,z
4804403,,0,0.0,0.0,0.0,0.0


In [23]:
#row number is 4804403 and now we will drop it
data = data.drop(4804403)

In [24]:
corr = data.corr()
corr

,Timestamp,x,y,z
Timestamp,1.000000,0.013351,-0.154436,-0.111103
x,0.013351,1.000000,0.025044,-0.111638
y,-0.154436,0.025044,1.000000,0.113206
z,-0.111103,-0.111638,0.113206,1.000000


In [25]:
data

,Subject-id,Activity Label,Timestamp,x,y,z
0,1600,walking,2.522077e+14,-0.364761,8.793503,1.055084
1,1600,walking,2.522077e+14,-0.879730,9.768784,1.016998
2,1600,walking,2.522078e+14,2.001495,11.109070,2.619156
3,1600,walking,2.522078e+14,0.450623,12.651642,0.184555
4,1600,walking,2.522079e+14,-2.164352,13.928436,-4.422485
...,...,...,...,...,...,...
1369653,1619,dribbling,1.739443e+14,-0.282295,-0.335558,-0.039415
1369654,1619,dribbling,1.739443e+14,-0.524110,-0.634898,-0.059655
1369655,1619,dribbling,1.739443e+14,-0.532632,-0.763795,0.103331
1369656,1619,dribbling,1.739444e+14,-0.255663,-0.641289,0.286556


## Data Balancing

In [26]:
data['Activity Label'].unique()

array(['walking', 'jogging', 'stairs', 'sitting', 'standing', 'typing',
       'brushing teeth', 'drinking soup', 'eating chips', 'eating pasta',
       'drinking', 'eating sandwich', 'kicking', 'catch', 'dribbling',
       'writing', 'clapping', 'folding'], dtype=object)

In [27]:
data['Subject-id'].value_counts().min()

64298

In [28]:
data['Subject-id'].value_counts()

1614    247195
1603    163040
1613    162884
1601    162650
1605    162462
1641    160798
1646    160792
1648    160791
1645    160783
1608    160772
1644    160771
1627    155101
1616    154571
1607    153391
1643    151834
1602    149176
1610    145114
1619    145004
1642    142929
1629    139346
1635    128852
1615    128563
1611    128561
1600    128558
1612    128558
1606    128558
1604    128557
1617    128551
1609    122285
1618    121453
1625    114850
1620    111026
1628    108362
1623     91143
1633     85955
1650     85811
1649     81975
1647     81887
1621     81688
1626     75034
1650     74968
1640     64845
1639     64840
1638     64830
1637     64801
1631     64315
1634     64309
1636     64308
1622     64308
1632     64306
1624     64302
1630     64298
Name: Subject-id, dtype: int64

In [29]:
data['Subject-id'].unique()

array([1600, 1601, 1602, 1603, 1604, 1605, 1606, 1607, 1608, 1609, 1610,
       1611, 1612, 1613, 1614, 1615, 1616, 1617, 1618, 1619, 1620, 1621,
       1622, 1623, 1624, 1625, 1626, 1627, 1628, 1629, 1630, 1631, 1632,
       1633, 1634, 1635, 1636, 1637, 1638, 1639, 1640, 1641, 1642, 1643,
       1644, 1645, 1646, 1647, 1648, 1649, 1650, '1650'], dtype=object)

In [30]:
a = data[data['Subject-id']== 1600].head(64298).copy()
b = data[data['Subject-id']== 1601].head(64298).copy()
c = data[data['Subject-id']== 1602].head(64298).copy()
d = data[data['Subject-id']== 1603].head(64298).copy()
e = data[data['Subject-id']== 1604].head(64298).copy()
f = data[data['Subject-id']== 1605].head(64298).copy()
g = data[data['Subject-id']== 1606].head(64298).copy()
h = data[data['Subject-id']== 1607].head(64298).copy()
i = data[data['Subject-id']== 1608].head(64298).copy()
j = data[data['Subject-id']== 1609].head(64298).copy()
k = data[data['Subject-id']== 1610].head(64298).copy()
l = data[data['Subject-id']== 1611].head(64298).copy()
m = data[data['Subject-id']== 1612].head(64298).copy()
n = data[data['Subject-id']== 1613].head(64298).copy()
o = data[data['Subject-id']== 1614].head(64298).copy()
p = data[data['Subject-id']== 1615].head(64298).copy()
q = data[data['Subject-id']== 1616].head(64298).copy()
r = data[data['Subject-id']== 1617].head(64298).copy()
s = data[data['Subject-id']== 1618].head(64298).copy()
t = data[data['Subject-id']== 1619].head(64298).copy()
u = data[data['Subject-id']== 1620].head(64298).copy()
v = data[data['Subject-id']== 1621].head(64298).copy()
w = data[data['Subject-id']== 1622].head(64298).copy()
x = data[data['Subject-id']== 1623].head(64298).copy()
y = data[data['Subject-id']== 1624].head(64298).copy()
z = data[data['Subject-id']== 1625].head(64298).copy()
aa = data[data['Subject-id']== 1626].head(64298).copy()
bb = data[data['Subject-id']== 1627].head(64298).copy()
cc = data[data['Subject-id']== 1628].head(64298).copy()
dd = data[data['Subject-id']== 1629].head(64298).copy()
ee = data[data['Subject-id']== 1630].head(64298).copy()
ff = data[data['Subject-id']== 1631].head(64298).copy()
gg = data[data['Subject-id']== 1632].head(64298).copy()
hh = data[data['Subject-id']== 1633].head(64298).copy()
ii = data[data['Subject-id']== 1634].head(64298).copy()
jj = data[data['Subject-id']== 1635].head(64298).copy()
kk = data[data['Subject-id']== 1636].head(64298).copy()
ll = data[data['Subject-id']== 1637].head(64298).copy()
mm = data[data['Subject-id']== 1638].head(64298).copy()
nn = data[data['Subject-id']== 1639].head(64298).copy()
oo = data[data['Subject-id']== 1640].head(64298).copy()
pp = data[data['Subject-id']== 1641].head(64298).copy()
qq = data[data['Subject-id']== 1642].head(64298).copy()
rr = data[data['Subject-id']== 1643].head(64298).copy()
ss = data[data['Subject-id']== 1644].head(64298).copy()
tt = data[data['Subject-id']== 1645].head(64298).copy()
uu = data[data['Subject-id']== 1646].head(64298).copy()
vv = data[data['Subject-id']== 1647].head(64298).copy()
ww = data[data['Subject-id']== 1648].head(64298).copy()
xx = data[data['Subject-id']== 1649].head(64298).copy()
yy = data[data['Subject-id']== 1650].head(64298).copy()
zz = data[data['Subject-id']== '1650'].head(64298).copy()

In [31]:
ddata = pd.DataFrame()

In [32]:
ddata = ddata.append([a,b,c,d,e,f,g,h,i,j,k,l,m,n,o,p,q,r,s,t,u,v,w,x,y,z,aa,bb,cc,dd,ee,ff,gg,hh,ii,jj,kk,ll,mm,nn,oo,pp,qq,rr,ss,tt,uu,vv,ww,xx,yy,zz]) 

In [33]:
ddata['Subject-id'] = ddata['Subject-id'].replace({'1650': 1651})

In [34]:
ddata['Subject-id'].unique()

array([1600, 1601, 1602, 1603, 1604, 1605, 1606, 1607, 1608, 1609, 1610,
       1611, 1612, 1613, 1614, 1615, 1616, 1617, 1618, 1619, 1620, 1621,
       1622, 1623, 1624, 1625, 1626, 1627, 1628, 1629, 1630, 1631, 1632,
       1633, 1634, 1635, 1636, 1637, 1638, 1639, 1640, 1641, 1642, 1643,
       1644, 1645, 1646, 1647, 1648, 1649, 1650, 1651], dtype=int64)

In [35]:
ddata.shape

(3343496, 6)

In [36]:
ddata['Activity Label'].unique()

array(['walking', 'jogging', 'stairs', 'sitting', 'standing', 'typing',
       'brushing teeth', 'drinking soup', 'eating chips', 'eating pasta',
       'drinking', 'eating sandwich', 'kicking', 'catch', 'dribbling',
       'writing', 'clapping', 'folding'], dtype=object)

In [37]:
ddata['Activity Label'].value_counts()

walking            282739
brushing teeth     266036
standing           266031
jogging            264836
sitting            261020
stairs             252072
typing             242784
drinking soup      206861
eating chips       185418
drinking           171299
kicking            162058
eating pasta       160273
eating sandwich    158681
catch              137690
dribbling          101533
writing             81555
clapping            73332
folding             69278
Name: Activity Label, dtype: int64

In [38]:
drinking = ddata[ddata['Activity Label']== 'drinking'].head(69278).copy()
walking = ddata[ddata['Activity Label']== 'walking'].head(69278).copy()
kicking = ddata[ddata['Activity Label']== 'kicking'].head(69278).copy()
drinking_soup = ddata[ddata['Activity Label']== 'drinking soup'].head(69278).copy()
brushing_teeth = ddata[ddata['Activity Label']== 'brushing teeth'].head(69278).copy()
standing = ddata[ddata['Activity Label']== 'standing'].head(69278).copy()
catch = ddata[ddata['Activity Label']== 'catch'].head(69278).copy()
dribbling = ddata[ddata['Activity Label']== 'dribbling'].head(69278).copy()
eating_sandwich = ddata[ddata['Activity Label']== 'eating sandwich'].head(69278).copy()
sitting = ddata[ddata['Activity Label']== 'sitting'].head(69278).copy()
clapping = ddata[ddata['Activity Label']== 'clapping'].head(69278).copy()
eating_chips = ddata[ddata['Activity Label']== 'eating chips'].head(69278).copy()
jogging = ddata[ddata['Activity Label']== 'jogging'].head(69278).copy()
folding = ddata[ddata['Activity Label']== 'folding'].head(69278).copy()
writing = ddata[ddata['Activity Label']== 'writing'].head(69278).copy()
stairs = ddata[ddata['Activity Label']== 'stairs'].head(69278).copy()
eating_pasta = ddata[ddata['Activity Label']== 'eating pasta'].head(69278).copy()
typing = ddata[ddata['Activity Label']== 'typing'].head(69278).copy()

In [39]:
bdata = pd.DataFrame()

In [40]:
bdata = bdata.append([walking ,jogging ,stairs,sitting,standing,typing,brushing_teeth,drinking_soup,eating_chips,eating_pasta,drinking,eating_sandwich,kicking,catch,dribbling,writing,clapping,folding]) 

In [41]:
bdata.shape

(1247004, 6)

In [42]:
bdata

,Subject-id,Activity Label,Timestamp,x,y,z
0,1600,walking,2.522077e+14,-0.364761,8.793503,1.055084
1,1600,walking,2.522077e+14,-0.879730,9.768784,1.016998
2,1600,walking,2.522078e+14,2.001495,11.109070,2.619156
3,1600,walking,2.522078e+14,0.450623,12.651642,0.184555
4,1600,walking,2.522079e+14,-2.164352,13.928436,-4.422485
...,...,...,...,...,...,...
3380512,1640,folding,1.705722e+14,3.679439,-9.209895,-4.233397
3380513,1640,folding,1.705723e+14,4.108001,-9.133281,-3.814412
3380514,1640,folding,1.705723e+14,4.220529,-8.513183,-3.539079
3380515,1640,folding,1.705724e+14,3.732111,-7.641693,-3.136853


In [43]:
#plot frequnecy
# Fs= 20 #since the data is recorded at frequnecy of 200Hz
# def plot_axis(ax, x, y, title):
#     ax.plot(x, y,'g')
#     ax.set_title(title)
#     ax.xaxis.set_visible(False)
#     ax.set_ylim([min(y) - np.std(y), max(y) + np.std(y)])
#     ax.set_xlim([min(x), max(x)])
#     ax.grid(True)

# def plot_activity(activity,bdata):
#     fig, (ax0, ax1, ax2) = plt.subplots(nrows = 3, figsize = (15, 7), sharex = True)
#     plot_axis(ax0, bdata['Timestamp'], bdata['x'], 'x')
#     plot_axis(ax1, bdata['Timestamp'], bdata['y'], 'y')
#     plot_axis(ax2, bdata['Timestamp'], bdata['z'], 'z')
#     plt.subplots_adjust(hspace=0.2)
#     fig.suptitle(activity)
#     plt.subplots_adjust(top=0.90)
#     plt.show()

# for activity in activities:
#     data_for_plot = bdata[(bdata['Activity Label'] == activity)][:Fs*10]
#     plot_activity(activity,data_for_plot)

In [44]:
#dropping the redundant columns
bdata = bdata.drop(['Subject-id'] , axis=1)
bdata.head()

,Activity Label,Timestamp,x,y,z
0,walking,2.522077e+14,-0.364761,8.793503,1.055084
1,walking,2.522077e+14,-0.879730,9.768784,1.016998
2,walking,2.522078e+14,2.001495,11.109070,2.619156
3,walking,2.522078e+14,0.450623,12.651642,0.184555
4,walking,2.522079e+14,-2.164352,13.928436,-4.422485


In [45]:
bdata.shape

(1247004, 5)

In [46]:
#activity label is of categorical type
#now we will use label encoder to encode this data
#labels are given to activity in alphabetical order
label = LabelEncoder()
bdata['Elabel'] = label.fit_transform(bdata['Activity Label'])
bdata.head()

,Activity Label,Timestamp,x,y,z,Elabel
0,walking,2.522077e+14,-0.364761,8.793503,1.055084,16
1,walking,2.522077e+14,-0.879730,9.768784,1.016998,16
2,walking,2.522078e+14,2.001495,11.109070,2.619156,16
3,walking,2.522078e+14,0.450623,12.651642,0.184555,16
4,walking,2.522079e+14,-2.164352,13.928436,-4.422485,16


In [47]:
label.classes_

array(['brushing teeth', 'catch', 'clapping', 'dribbling', 'drinking',
       'drinking soup', 'eating chips', 'eating pasta', 'eating sandwich',
       'folding', 'jogging', 'kicking', 'sitting', 'stairs', 'standing',
       'typing', 'walking', 'writing'], dtype=object)

### standardization of data

In [54]:
x = bdata[['x','y','z']]
y = bdata['Elabel']
x,y

(                x          y         z
 0       -0.364761   8.793503  1.055084
 1       -0.879730   9.768784  1.016998
 2        2.001495  11.109070  2.619156
 3        0.450623  12.651642  0.184555
 4       -2.164352  13.928436 -4.422485
 ...           ...        ...       ...
 3380512  3.679439  -9.209895 -4.233397
 3380513  4.108001  -9.133281 -3.814412
 3380514  4.220529  -8.513183 -3.539079
 3380515  3.732111  -7.641693 -3.136853
 3380516  3.511845  -7.579444 -3.103334
 
 [1247004 rows x 3 columns],
 0          16
 1          16
 2          16
 3          16
 4          16
            ..
 3380512     9
 3380513     9
 3380514     9
 3380515     9
 3380516     9
 Name: Elabel, Length: 1247004, dtype: int32)

In [64]:
scaler = StandardScaler()
x= scaler.fit_transform(x)
scaled_x = pd.DataFrame(data=x, columns = ['x','y','z'])
scaled_x.head()

,x,y,z
0,-0.191700,1.100036,-0.160677
1,-0.286219,1.238546,-0.168645
2,0.242611,1.428894,0.166572
3,-0.042041,1.647971,-0.342816
4,-0.522003,1.829302,-1.306742


In [95]:
y= y.to_numpy()

In [133]:
y = y.reshape(-1,)

In [134]:
y.shape

(1247004,)

In [131]:
Ys = scaler.fit_transform(y)

In [132]:
scaled_x.shape, Ys.shape

((1247004, 3), (1247004, 1))

In [118]:
from sklearn.model_selection import train_test_split

In [136]:
trainX, testX, trainy, testy = train_test_split(scaled_x, y, test_size=0.33, random_state=42)

In [137]:
def define_models(models=dict()):
    models['knn'] = KNeighborsClassifier(n_neighbors=7)
    models['cart'] = DecisionTreeClassifier()
    models['svm'] = SVC()
    print('Defined %d models' % len(models))
    return models

In [141]:
def evaluate_models(trainX, trainy, testX, testy, models):
    results = dict()
    for name, model in models.items():
        # evaluate the model
        results[name] = evaluate_model(trainX, trainy, testX, testy, model)
    return results

In [ ]:
models = define_models()
results = evaluate_models(trainX, trainy, testX, testy, models)

Defined 3 models
